In [ ]:
import torch 
import numpy as np
import os
import sys
sys.path.append("..")
from src.model import Encoder, Decoder
from src.dataset import ResidualDataset
from src.HuffmanCompression import HuffmanCoding
import pdb
from copy import copy
import time
%load_ext autoreload
%autoreload 2

In [ ]:
checkpoint_enc = '../checkpoint/test/encoder-epoch_49_iter_40550.pth'
checkpoint_dec = '../checkpoint/test/decoder-epoch_49_iter_40550.pth'
enc = Encoder()
dec = Decoder()
# load both models
enc.load_state_dict(torch.load(checkpoint_enc, map_location='cpu'))
dec.load_state_dict(torch.load(checkpoint_dec, map_location='cpu'))

In [ ]:
train_data_path = '../data'
device = torch.device('cpu')
dSet_train = ResidualDataset(train_data_path, 'final_test', device)
dataset_train = torch.utils.data.DataLoader(dSet_train,
                                            batch_size=1, shuffle=True,
                                            num_workers=0)

In [ ]:
bin_out_list = []
with torch.no_grad():
    for idx, sample in enumerate(dataset_train):
        model_input = sample['image']
        bin_out = enc(model_input).squeeze().numpy()
        bin_out_list.append(bin_out)
bin_out_arr = ((np.array(bin_out_list) + 1)/2).astype(np.uint8)
bin_out_arr.shape

In [ ]:
test_bin = bin_out_arr[10,:,:,:]
test_bin = test_bin[np.newaxis]
# test_bin = bin_out_arr
print(test_bin.shape)

In [ ]:
N,C,H,W = test_bin.shape
tile_x = 9
tile_y = 9
huffmap ={}

pH = pW = 0
if  H % tile_y != 0:
    pH = tile_y - H % tile_y
if  W % tile_x != 0:
    pW = tile_x - W % tile_x


start_time = time.time()

test_bin_padded = np.pad(test_bin, ((0,0), (0,0), (0, pH), (0, pW)))
# print(test_bin_padded.shape)

N, C, H, W = test_bin_padded.shape

for n in range(N):
    for c in range(C):
        for i in range(H // tile_x):
            ii = i*tile_y
            for j in range(W // tile_y):
                jj = j*tile_x
                tile = test_bin_padded[n,c, ii:ii+tile_y, jj:jj+tile_x].flatten()
                s = ""
                for ele in tile:
                    s += str(ele)
                huffmap[s] = huffmap.get(s, 0) + 1
                
h = HuffmanCoding()
h.make_heap(huffmap)
h.merge_nodes()
h.make_codes()

print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
# tot_bits = 0
entropy =0.
avg = 0
total_counts = 0.
codes = h.codes
tot_bits = 0
node_count = 0
tree_size = 0
for key in huffmap.keys():
    tot_bits += huffmap[key] * len(codes[key])
    total_counts += huffmap[key]
    node_count += 1
print("Total bits in data: {}, Total counts: {}".format(N*C*H*W, total_counts))
print("Total bits needed: {}".format(tot_bits))
for key in huffmap.keys():
    avg += huffmap[key] * len(codes[key])/total_counts
    p = float(huffmap[key])/total_counts
    entropy += -(p * np.log2(p))
print("Entropy: {}".format(entropy))
print("Avg code work length: {}".format(avg))
print("How close are we to Entropy :{} ".format(entropy/avg))
tree_size = (2 + tile_x*tile_y) * node_count - 1
print("Node Count: {}, Size of Codes: {}, Compression ratio in general: {}".format(node_count, tree_size, (tile_x*tile_y)/avg))
print("compression ratio includes padding: {}".format(216000/tot_bits))
print("compression ratio includes padding and tree(per frame): {}".format(216000/(tot_bits+tree_size)))



In [ ]:
For N = 86
(runtime(s), size of codes(B))
tile 5x5  : 25.3414, 20971624
tile 7x7  : 21.0083, 20971624
tile 9x9  : 19.9685, 10485872
tile 11x11: 19.2659, 5242984
tile 13x13: 17.6710, 5242984
    
For N = 1
tile 5x5  : 0.3230, 295016, 1.9522
tile 7x7  : 0.2490, 147568, 3.6048
tile 9x9  : 0.2300, 73832, 6.9171
tile 11x11: 0.2236, 73832, 8.8152
tile 13x13: 0.2131, 73832, 13.2923